In [1]:
import os
import httpx
import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import requests
import numpy as np
import import_ipynb
import ollama
from evaluate import load
from rouge import Rouge


In [2]:
import import_ipynb
import importlib
import Eval_Metrics
eval_metrics = Eval_Metrics.Evaluation_Metrics()
# importlib.reload(Eval_Metrics)
# eval_metrics = Eval_Metrics.Evaluation_Metrics()

### Getting Ranking List Prompt Dataset to be used in Ranking List LLM Evaluation

In [5]:
prompt_examples_df = pd.read_excel(r"C:\Users\flame\CP5105-AI-Powered-Product-and-Service-Review-Summarizer-and-Ranking-System\BackEnd\All Prompt Examples\Ranking List Prompt Dataset.xlsx")


In [6]:
prompt_examples_df['Modified_Multiple_Generated_Summaries'] = prompt_examples_df['Modified_Multiple_Generated_Summaries'].map(lambda a: a.replace('\n',' '))

In [7]:
prompt_examples_df.columns

Index(['Query', 'Modified_1_Generated_Summary',
       'Modified_Multiple_Generated_Summaries', 'Model_Ranking_List_Output',
       'Model_Ranking_List_for_Manual_Checking', 'Prompt'],
      dtype='object')

#### Getting Data to be used as Prompt Examples from dataset

In [8]:
promptengin_df = prompt_examples_df[prompt_examples_df['Prompt']=='Use for Prompt'].reset_index(drop=True)
promptengin_df

,Query,Modified_1_Generated_Summary,Modified_Multiple_Generated_Summaries,Model_Ranking_List_Output,Model_Ranking_List_for_Manual_Checking,Prompt
0,What are the worst mobile phone games released...,The gaming industry has been flooded with new ...,Article #1: The gaming industry has been flood...,Here is the ranking list for the worst mobile ...,1: Robot Rebellion\n2. Galactic Conquest\nRest...,Use for Prompt
1,What are the top 5 best sci-fi movies of all t...,This is a question that has been debated by fi...,Article #1: This is a question that has been ...,Here is the ranking list for the top 5 best sc...,1: Blade Runner\n2: The Matrix\n3: A Space Ody...,Use for Prompt
2,What are the 3 worst games consoles of all time?,The Atari Jaguar is often cited as one of the ...,Article #1: The Atari Jaguar is often cited a...,Here is the ranking list for the 3 worst games...,"1,2: Atari Jaguar, Nintendo Virtual Boy\n3: Se...",Use for Prompt
3,What are the 5 worst board games?,Determining the worst board games can be subje...,Article #1: Determining the worst board games ...,Here is the ranking list for the 5 worst board...,1: Monikers\n2: Don’t Wake Daddy\n3: Battleshi...,Use for Prompt
4,What are the best printer models in 2024?,Choosing the right printer can be overwhelming...,Article #1: Choosing the right printer can be ...,Here is the ranking list for the best printer ...,1. Epson Expression Photo HD XP-15000\n2. Xero...,Use for Prompt


#### Getting Data to be used for Ranking List LLM testing from dataset

In [10]:
testprompt_df = prompt_examples_df[prompt_examples_df['Prompt'].isna()==True].reset_index(drop=True)
testprompt_df

,Query,Modified_1_Generated_Summary,Modified_Multiple_Generated_Summaries,Model_Ranking_List_Output,Model_Ranking_List_for_Manual_Checking,Prompt
0,What are the top car models in 2024?,The automotive industry is constantly evolving...,Article #1: The automotive industry is constan...,Here is the ranking list for the top car model...,"1,2: Toyota Camry , Honda Civic Type R\n3,4: H...",NaN
1,What are the 3 worst laptop brands in 2024?,The ranking of laptop brands can vary dependin...,Article #1: The ranking of laptop brands can v...,Here is the ranking list for the 3 worst lapto...,1: Acer\n2: Lenovo\n3: Asus,NaN
2,What are the cheapest but decent laptop models...,"When it comes to budget-friendly laptops, ther...",Article #1: When it comes to budget-friendly l...,Here is the ranking list for the top budget la...,"1: Lenovo IdeaPad 330S\n2,3, 4 ,5: Acer Aspire...",NaN
3,What are the top sunscreen brands?,Sunscreen is a vital part of our daily skincar...,Article #1: Sunscreen is a vital part of our ...,Here is the ranking list for the top sunscreen...,"1. Neutrogena\n2. Hawaiian Tropic\n3,4,5: La R...",NaN
4,What are the most interesting and top rated bo...,The year 2024 is expected to bring a diverse r...,Article #1: The year 2024 is expected to bring...,Here is the ranking list for the most interest...,"1,2: The It Girl, The Memory Keeper\'s Daughte...",NaN
5,What are the most hated and worst books releas...,"Based on user ratings on Goodreads, some of th...",Article #1: Based on user ratings on Goodreads...,Here is the ranking list for the most hated an...,"1,2: No Going Back , Crown of Starlight\nRest ...",NaN
6,What are the 5 worst horror movies of all time?,The assessment of the worst horror movies is s...,Article #1: The assessment of the worst horror...,Here is the ranking list for the 5 worst horro...,1: Devil\n2: Birdemic: Shock and Terror (2010)...,NaN
7,What are the 3 best games consoles of all time?,"Determining the ""best"" gaming consoles is subj...","Article #1: Determining the ""best"" gaming cons...",Here is the ranking list for the top 3 best ga...,"1,2: PlayStation 2 (PS2) , Nintendo Entertainm...",NaN
8,What are the 5 best board games for children?,Choosing the right board game can be a challen...,Article #1: Choosing the right board game can ...,Here is the ranking list for the 5 best board ...,"1,2,3,4 : Candy Land, Chutes and Ladders, Mono...",NaN
9,What are the worst mobile phones in 2024?,The ranking of the worst mobile phones can be ...,Article #1: The ranking of the worst mobile ph...,Here is the ranking list for the worst mobile ...,"1. Motorola Defy Pro X\n2,3: HTC One M7, Samsu...",NaN


#### Getting the other half of the test data that Ranking List LLM Evaluation Part 1 (1st half) SCT vs SLT , Zero Shot vs Few (3) Shot Prompting did not use

In [11]:
snippet_prompt_df = testprompt_df[:5].reset_index(drop=True)
snippet_prompt_df

,Query,Modified_1_Generated_Summary,Modified_Multiple_Generated_Summaries,Model_Ranking_List_Output,Model_Ranking_List_for_Manual_Checking,Prompt
0,What are the top car models in 2024?,The automotive industry is constantly evolving...,Article #1: The automotive industry is constan...,Here is the ranking list for the top car model...,"1,2: Toyota Camry , Honda Civic Type R\n3,4: H...",NaN
1,What are the 3 worst laptop brands in 2024?,The ranking of laptop brands can vary dependin...,Article #1: The ranking of laptop brands can v...,Here is the ranking list for the 3 worst lapto...,1: Acer\n2: Lenovo\n3: Asus,NaN
2,What are the cheapest but decent laptop models...,"When it comes to budget-friendly laptops, ther...",Article #1: When it comes to budget-friendly l...,Here is the ranking list for the top budget la...,"1: Lenovo IdeaPad 330S\n2,3, 4 ,5: Acer Aspire...",NaN
3,What are the top sunscreen brands?,Sunscreen is a vital part of our daily skincar...,Article #1: Sunscreen is a vital part of our ...,Here is the ranking list for the top sunscreen...,"1. Neutrogena\n2. Hawaiian Tropic\n3,4,5: La R...",NaN
4,What are the most interesting and top rated bo...,The year 2024 is expected to bring a diverse r...,Article #1: The year 2024 is expected to bring...,Here is the ranking list for the most interest...,"1,2: The It Girl, The Memory Keeper\'s Daughte...",NaN


#### Ranking List response generation function for SLT text input

In [12]:
def MULTI_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered list from the following ARTICLEs: ' 
            + text,
        },
        # {
        #     'role': 'assistant',
        #     'content': f'{query}'     
        # }
        ]
        # options={
        #     'temperature':0.1
        # }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for SCT text input

In [13]:
def SINGLE_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered list from the following text input: ' 
            + text,
        },
        # {
        #     'role': 'assistant',
        #     'content': f'{query}'     
        # }
        ]
        # options={
        #     'temperature':0.1
        # }
        )
    
    
    report = response['message']['content']
    
    return report

##### Response DataFrame Initialization

In [14]:
zero_shot_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','avg_score'])

In [1]:
query =''

##### Chosen LLM

In [15]:
model = "llama3.1:latest"

### Zero Shot Prompt

In [17]:
def zero_shot_prompt_MULTI(query):
        zero_shot_prompt =  f'''You are a reviewer generating an ordered list based on {query} only. \
                Each item in the ordered list should be ordered from item that appeared most amount of times across all the ARTICLEs to the item that appeared least amount of times across all the ARTICLEs. \
                Each item in the ordered list should have the item name and the item details from the articles. \
                Bold each item on the ranking list in the output.\
                To make text bold, enclose it with double asterisks (**). Example: **text** becomes bold.\
                Do not include article names in the ordered list.\
                '''
        return zero_shot_prompt

In [18]:
def zero_shot_prompt_SINGLE(query):
        zero_shot_prompt =  f'''You are a reviewer generating an ordered list based on {query} only. \
                Each item in the ordered list should be ordered from item that appeared most amount of times in the user's text input to the item that appeared least amount of times in the user's text input. \
                Each item in the ordered list should have the item name and the item details from the user's text input. \
                Bold each item on the ranking list in the output.\
                To make text bold, enclose it with double asterisks (**). Example: **text** becomes bold.\
                '''
        return zero_shot_prompt

### Few (3) Shot Prompt

In [19]:
def few3_shot_prompt_MULTI(query):
        few3_shot_prompt =  f'''You are a reviewer generating an ordered list based on {query} only. \
                Each item in the ordered list should be ordered from item that appeared most amount of times across all the ARTICLEs to the item that appeared least amount of times across all the ARTICLEs. \
                Each item in the ordered list should have the item name and the item details from the articles. \
                Bold each item on the ranking list in the output.\
                To make text bold, enclose it with double asterisks (**). Example: **text** becomes bold.\
                Do not include article names in the ordered list.\
                ---\
                <EXAMPLE>\
                Prompt: In the context of "What are the top 5 best sci-fi movies of all time?" , generate an ordered list from the following ARTICLEs: Article #1: 
                This is a question that has been debated by film enthusiasts for years, with different opinions and rankings depending on personal taste and preferences. However, based on critical acclaim, popularity, and impact on popular culture, here are five of the most highly-regarded science fiction films of all time.   Blade Runner (1982) directed by Ridley Scott is a groundbreaking sci-fi classic that has been widely acclaimed for its thought-provoking themes and visually stunning depiction of a dystopian future. The movie follows Rick Deckard, a bounty hunter tasked with tracking down advanced androids known as replicants who have escaped and are seeking to extend their short lifespan. Set in a polluted and overcrowded Los Angeles, the film explores complex questions about humanity, identity, and what it means to be alive.  Another iconic sci-fi film is The Matrix (1999) directed by the Wachowskis. This innovative movie revolutionized visual effects and special effects with its intricate "bullet time" sequences and stunning "green screen" work. The story revolves around Neo, a computer hacker who discovers that his entire life has been simulated within a complex virtual reality program created by intelligent machines to subdue humanity. As Neo delves deeper into the mysteries of this world, he must choose between his loyalty to humanity and his newfound connection with the machines.  Inception (2010) directed by Christopher Nolan is another thought-provoking sci-fi epic that has captivated audiences worldwide with its intricate plot and complex themes. The movie follows Cobb, a skilled thief who specializes in entering people\'s dreams and stealing their secrets. With the help of his team, Cobb embarks on a perilous mission to plant an idea in someone\'s mind instead of stealing it, but he soon finds himself trapped in a labyrinthine world where the boundaries between reality and fantasy blur.  Stanley Kubrick\'s 2001: A Space Odyssey (1968) is widely regarded as one of the greatest films of all time, and its sci-fi elements have had a profound influence on popular culture. The movie follows a team of astronauts on a mission to Jupiter, where they encounter a mysterious black monolith that holds the key to humanity\'s evolution and destiny in space. As the crew delves deeper into the mysteries of this monolith, they must confront their own existence and the possibility of extraterrestrial life.  The Last Jedi (2017) directed by Rian Johnson is often considered one of the greatest sci-fi movies of all time due to its bold storytelling, memorable characters, and subversive themes. The film takes place after the events of The Force Awakens and follows Rey as she embarks on a journey to find Luke Skywalker, who has gone into exile after his failure to stop Kylo Ren from destroying the Resistance\'s fleet. As Rey navigates this treacherous world, she must confront her own destiny, the true nature of the Force, and the secrets that have been hidden for centuries.  In conclusion, these five sci-fi movies are widely regarded as some of the greatest films ever made due to their thought-provoking themes, visually stunning depiction of a futuristic world, and profound impact on popular culture. Whether you\'re a fan of classic dystopian futures or cutting-edge space opera, these movies will transport you to new worlds and challenge your perceptions about humanity\'s place in the universe.  These five sci-fi films have stood the test of time due to their exceptional storytelling, memorable characters, and subversive themes that continue to captivate audiences worldwide. From Blade Runner\'s exploration of what it means to be human to The Matrix\'s innovative visual effects and complex world-building, each of these movies has contributed significantly to the sci-fi genre.
                Article #2: 
                This question has been debated by film enthusiasts and critics for decades, with various opinions on which films truly stand out as the greatest sci-fi movies ever made.  The Matrix (1999) directed by The Wachowskis is one of the most iconic sci-fi movies in recent history. It tells the story of Neo (Keanu Reeves), a computer hacker who discovers that his entire life has been a simulation created by intelligent machines to distract humans while their bodies are harvested for energy. With its innovative special effects, intricate storyline, and thought-provoking themes, The Matrix is widely regarded as one of the greatest sci-fi films ever made.  Blade Runner (1982) directed by Ridley Scott is another classic sci-fi movie that explores what it means to be human. Set in a dystopian future where synthetic beings known as replicants are hunted by special police officers called blade runners, the film delves into complex questions about artificial intelligence, empathy, and humanity. With its atmospheric visuals, rich themes, and memorable characters, Blade Runner has become a cult classic among sci-fi fans.  2001: A Space Odyssey (1968) directed by Stanley Kubrick is often cited as one of the most influential films in science fiction history. This groundbreaking film tells the story of a team of astronauts who embark on a mission to Jupiter, where they encounter a mysterious alien monolith that holds the key to humanity's evolution. With its groundbreaking special effects, philosophical themes, and enigmatic ending, it continues to inspire new generations of filmmakers and sci-fi enthusiasts.  Alien (1979) directed by Ridley Scott is another iconic sci-fi movie that has become a staple of the genre. The film tells the story of a crew of space explorers who are stalked and killed one by one by a deadly alien creature on a distant planet. With its claustrophobic tension, memorable characters, and groundbreaking visual effects, Alien has become one of the most beloved sci-fi horror films of all time.  In conclusion, the top 5 best sci-fi movies of all time include The Matrix (1999), Blade Runner (1982), 2001: A Space Odyssey (1968), Alien (1979), and Star Wars: Episode IV - A New Hope (1977). These films have captivated audiences with their imaginative storylines, memorable characters, and groundbreaking special effects, solidifying their places in the pantheon of science fiction cinema.
                Article #3: 
                This question is often subjective, as opinions may vary depending on personal taste and preferences. However, here is a summary of some of the most iconic and influential science fiction films that have made it to various "best of" lists:  One of the most popular and enduring sci-fi movies of all time is Stanley Kubrick\'s "2001: A Space Odyssey" (1968). Directed by Kubrick and written by Arthur C. Clarke, this groundbreaking film follows a team of astronauts on a mission to Jupiter and explores themes of human evolution, technology, and existentialism. The movie\'s slow pace and philosophical undertones were considered revolutionary at the time of its release.  Another classic sci-fi film that has stood the test of time is Ridley Scott\'s "Blade Runner" (1982). Set in a dystopian future where synthetic humans are hunted by special police officers known as blade runners, this visually stunning movie explores complex themes such as humanity, artificial intelligence, and what it means to be alive. The film\'s atmospheric and introspective tone has influenced countless other sci-fi movies and TV shows.  Martin Scorsese\'s "Alien" (1979) is a seminal work of science fiction that revolutionized the horror genre with its terrifying alien creatures and claustrophobic setting. Directed by Ridley Scott, this movie follows a crew of space explorers who are stalked and killed by a deadly extraterrestrial entity, leading to a thrilling cat-and-mouse chase through the spaceship\'s corridors.  Steven Spielberg\'s "E.T. the Extra-Terrestrial" (1982) is often cited as one of the greatest family movies of all time, but it also explores complex themes such as first contact, xenophobia, and the power of childhood innocence. The movie follows a young boy named Elliott who befriends an alien who has been left behind on Earth and must help him return home.  The Matrix (1999) is another influential sci-fi movie that redefined the genre with its innovative special effects, intricate storyline, and thought-provoking themes. Directed by the Wachowskis, this movie follows a computer hacker named Neo who discovers that his entire life has been a simulated reality created by sentient machines to distract humans while their bodies are harvested for energy.  These five movies have all had a significant impact on popular culture and continue to be widely discussed and debated by fans of science fiction. While opinions may vary, they are generally regarded as some of the greatest sci-fi films of all time due to their thought-provoking themes, visually stunning cinematography, and enduring influence on the genre.  In conclusion, these five movies have captured the imagination of audiences worldwide with their complex characters, engaging storylines, and exploration of deeper themes. They continue to be celebrated as some of the greatest sci-fi films of all time and remain essential viewing for fans of science fiction and anyone interested in exploring new ideas and perspectives.
                \
                Response: Here is the ranking list for the top 5 best sci-fi movies of all time with their summaries from most to least:
                \n
                **#1. Blade Runner:**
                'Blade Runner, directed by Ridley Scott in 1982, is a groundbreaking and thought-provoking sci-fi classic that explores complex themes about humanity, identity, and what it means to be alive. Set in a dystopian future where synthetic beings known as replicants are hunted by special police officers called blade runners, the film delves into questions about artificial intelligence, empathy, and humanity through its atmospheric visuals, rich themes, and memorable characters, ultimately becoming a cult classic among sci-fi fans that has influenced numerous other movies and TV shows.'
                \n
                **#2. The Matrix: **
                'The Matrix (1999), directed by the Wachowskis, is an iconic and influential sci-fi movie that revolutionized visual effects with its "bullet time" sequences and "green screen" work. The film tells the story of Neo, a computer hacker who discovers his life has been simulated within a virtual reality program created by intelligent machines to subdue humanity, forcing him to choose between loyalty to humans and connection with the machines. With its innovative special effects, intricate storyline, and thought-provoking themes, The Matrix is widely regarded as one of the greatest sci-fi films ever made, redefining the genre with its groundbreaking storytelling and visuals.'
                \n
                **#3. 2001: A Space Odyssey: **
                "Stanley Kubrick's 2001: A Space Odyssey (1968) is widely regarded as one of the greatest films of all time and has had a profound influence on popular culture, particularly in the science fiction genre. The film follows a team of astronauts on a mission to Jupiter who encounter a mysterious alien monolith that holds the key to humanity's evolution and destiny, prompting them to confront their own existence and the possibility of extraterrestrial life. With its groundbreaking special effects, philosophical themes, and enigmatic ending, 2001: A Space Odyssey continues to inspire new generations of filmmakers and sci-fi enthusiasts with its thought-provoking exploration of human evolution, technology, and existentialism."
                \n
                **#4. Alien: **
                'The iconic sci-fi horror film Alien, directed by Ridley Scott in 1979, tells the story of a crew of space explorers being stalked and killed one by one by a deadly alien creature on a distant planet. With its groundbreaking visual effects, memorable characters, and claustrophobic tension, Alien has become a seminal work of science fiction that revolutionized the horror genre with its terrifying alien creatures and eerie setting.'
                \n
                **#5. The Last Jedi: **
                "The Last Jedi, directed by Rian Johnson, is often considered one of the greatest sci-fi movies of all time due to its bold storytelling, memorable characters, and subversive themes. The film follows Rey's journey as she searches for Luke Skywalker, who has gone into hiding after his failure to stop Kylo Ren from destroying the Resistance's fleet, while also grappling with her own destiny and the mysteries of the Force."
                \n
                \n
                'The five sci-fi movies discussed are widely regarded as some of the greatest films ever made due to their thought-provoking themes, visually stunning depiction of a futuristic world, and profound impact on popular culture. These movies have captivated audiences with their imaginative storylines, memorable characters, and groundbreaking special effects, solidifying their places in the pantheon of science fiction cinema and continuing to be widely discussed and debated by fans of science fiction. Their complex world-building, exploration of deeper themes, and enduring influence on the genre have cemented their status as essential viewing for fans of science fiction and anyone interested in exploring new ideas and perspectives.'
                \
                \
                ---\
                <EXAMPLE>\
                Prompt: In the context of "What are the 3 worst games consoles of all time?" , generate an ordered list from the following ARTICLEs: Article #1: 
                The Atari Jaguar is often cited as one of the most disappointing consoles ever made, with its high price and lackluster lineup of games failing to attract a significant audience.  The Sega Dreamcast was another console that failed to gain traction in the market, despite its innovative controller design and online capabilities. Its short lifespan and limited support from third-party developers also contributed to its failure. The console is still remembered fondly by some enthusiasts for its quirky charm and exclusive games such as Sonic Adventure and Crazy Taxi.  The Nintendo Virtual Boy is widely regarded as one of the worst consoles ever made, with its poor graphics, lackluster lineup of games, and high price point making it a flop in the market. The console's failure was so severe that it led to a significant decline in Nintendo's stock value and a reevaluation of their hardware design strategies.  The Atari 7800 is another console often cited as one of the worst due to its poor design and lackluster lineup of games, with many critics feeling that it was little more than an improved version of the Atari 2600. The console's high price point and limited support from third-party developers also contributed to its failure in the market.  The Sega Nomad is often cited as one of the worst handheld consoles ever made due to its poor battery life, lackluster lineup of games, and high price point. Its design flaws and limited functionality made it unpopular among gamers, leading to a significant decline in sales for Sega.  The Nintendo 64DD was another console that failed to gain traction in the market, with its slow loading times, limited support from third-party developers, and lackluster lineup of games making it unpopular among gamers. Its short lifespan and poor marketing also contributed to its failure, ultimately leading to a loss for Nintendo.  The Sega SG-1000 is often cited as one of the worst consoles ever made due to its poor design, lackluster lineup of games, and high price point. Its limited functionality and compatibility issues made it unpopular among gamers, leading to a significant decline in sales for Sega.  Each of these consoles has been remembered fondly by some enthusiasts for their quirks and charm, but overall they are widely regarded as some of the worst consoles ever made due to their poor design, lackluster lineups of games, and limited functionality.
                Article #2: 
                When evaluating the worst gaming consoles ever made, it's essential to consider several factors such as their technical issues, poor game library, and overall impact on the gaming industry.  One of the worst consoles is the Sega Dreamcast released in 1998. Although it received widespread critical acclaim for its innovative design and impressive games like Sonic Adventure and Crazy Taxi, it struggled financially due to a lack of third-party support and limited release in various countries. This led to a significant decline in sales, ultimately causing the console's demise.  Another contender is the Nintendo Virtual Boy released in 1995. Although touted as the successor to the Super Nintendo, this console failed miserably due to its poor game library and numerous technical issues. Its games were often criticized for their low resolution and lackluster graphics, which further exacerbated the console's failure.  Lastly, the Atari Jaguar, released in 1993, is also considered one of the worst consoles ever made. Despite being launched with significant hype, it suffered from a limited game library and numerous technical issues such as overheating and poor sound quality. Its failure was so catastrophic that it nearly bankrupted the company, leading to a massive decline in its market value.  The three consoles were unable to gain substantial market share or even stay afloat for an extended period. While some may argue that their failures led to improvements in future gaming technology and innovations, they undoubtedly remain among the worst console releases in history.
                Article #3: 
                The Atari Jaguar, Sega Saturn, and Nintendo Virtual Boy have been widely regarded as some of the worst game consoles ever released.  The Atari Jaguar was released in 1993 and failed miserably due to its high price tag, poor marketing, and lackluster library of games. It had a short lifespan, lasting only three years before being discontinued. The console's failure was attributed to its inability to compete with other popular consoles such as the Sega Genesis and Super Nintendo Entertainment System.  The Sega Saturn, released in 1994, was plagued by poor marketing and high development costs. Despite its impressive graphics capabilities, it failed to gain significant market share due to its limited game library and lack of third-party support. The console's failure led to a significant decline in Sega's fortunes, resulting in the company eventually abandoning the Saturn and shifting focus to other ventures.  The Nintendo Virtual Boy was released in 1995 as a pioneering attempt at virtual reality gaming but ultimately failed due to its high price, lackluster library of games, and limited lifespan. The console was criticized for its poor graphics, awkward design, and lack of appeal to mainstream gamers. Its failure led to Nintendo abandoning the concept of virtual reality gaming for several years.  In conclusion, these three consoles are often cited as some of the worst game consoles ever released due to their high prices, poor marketing, and limited libraries of games.
                \
                Response: Here is the ranking list for the 3 worst games consoles of all time with their summaries from the most to least bad:
                \n
                **#1. Atari Jaguar:**
                The Atari Jaguar is widely regarded as one of the most disappointing consoles ever made, released in 1993 with high hopes but ultimately failing due to a combination of factors including its high price tag, limited game library, and numerous technical issues. Its catastrophic failure nearly bankrupted the company and led to a significant decline in market value, forcing the console's discontinuation just three years after its release.
                \n
                **#2. Nintendo Virtual Boy:**
                The Nintendo Virtual Boy is widely regarded as one of the worst consoles ever made due to its poor graphics, limited game library, and high price point, which ultimately led to a significant decline in Nintendo's stock value and a reevaluation of their hardware design strategies. Released in 1995 as an attempt at virtual reality gaming, the console failed miserably due to its numerous technical issues, awkward design, and lack of appeal to mainstream gamers, causing Nintendo to abandon the concept of virtual reality gaming for several years.
                \n
                **#3. Sega Dreamcast:**
                The Sega Dreamcast was a console that failed to gain significant market traction, despite its innovative controller design and online capabilities, due to its short lifespan and limited third-party developer support. Despite receiving critical acclaim for its games such as Sonic Adventure and Crazy Taxi, it struggled financially and ultimately declined in sales, leading to its demise.
                \n
                The three consoles described in the text have been widely regarded as among the worst console releases in history due to their poor design, lackluster lineups of games, and limited functionality. Despite some potential benefits from their failures that led to innovations in future gaming technology, these consoles were unable to gain substantial market share or stay afloat for an extended period, ultimately being remembered more for their quirks and flaws than any redeeming qualities.
                \
                \
                ---\
                <EXAMPLE>\
                Prompt: In the context of "What are the best printer models in 2024?" , generate an ordered list from the following ARTICLEs: Article #1:
                Choosing the right printer can be overwhelming with so many options available, but here's a comprehensive guide to help you make an informed decision.  **Best Printer Models for Home Users**  1. **HP Envy 5055**: A compact and affordable all-in-one inkjet printer that prints up to 9 ppm in black and white and 7 ppm in color. 2. **Epson Expression Photo HD XP-15000**: A high-end photo printer that offers excellent print quality, Wi-Fi connectivity, and a large 3.7-inch touchscreen display. 3. **Canon PIXMA TS9521C**: A versatile all-in-one printer with wireless printing capabilities, CD/DVD printing, and a compact design.  **Best Printer Models for Office Users**  1. **HP LaserJet Pro M283fdw**: A reliable and high-speed monochrome laser printer that prints up to 30 ppm, has a large paper capacity, and supports mobile printing. 2. **Brother HL-L8360CDW**: A fast and feature-rich color laser printer with Wi-Fi connectivity, NFC, and a touchscreen display. 3. **Xerox VersaLink C405**: A powerful and versatile all-in-one printer that offers advanced security features, wireless connectivity, and a large paper capacity.  **Best Printer Models for Photo Printing**  1. **Epson SureColor P800**: A high-end photo printer with excellent print quality, Wi-Fi connectivity, and a range of paper sizes. 2. **Canon PIXMA PRO-200**: A professional-grade photo printer that offers exceptional print quality, wireless printing capabilities, and a compact design. 3. **HP Envy 7855**: A versatile all-in-one photo printer with wireless printing capabilities, CD/DVD printing, and a large 2.7-inch touchscreen display.  **Best Printer Models for Budget-Friendly Options**  1. **Canon PIXMA TS202**: A budget-friendly all-in-one inkjet printer that prints up to 8 ppm in black and white and 4 ppm in color. 2. **Epson Expression Home XP-4100**: A compact and affordable all-in-one inkjet printer with wireless printing capabilities and a large 2.7-inch touchscreen display. 3. **HP DeskJet Ink Advantage 2135**: A budget-friendly all-in-one inkjet printer that prints up to 7 ppm in black and white and 4 ppm in color.  **Key Features to Consider**  1. **Print Resolution**: Look for printers with high print resolutions (at least 4800 x 1200 dpi) for excellent photo printing quality. 2. **Wireless Connectivity**: Choose a printer with Wi-Fi or Bluetooth connectivity for convenient wireless printing from mobile devices. 3. **Paper Capacity**: Select a printer with a large paper capacity to minimize refills and reduce waste. 4. **Ink Costs**: Consider the cost of replacement ink cartridges when choosing a printer, as some models may be more expensive than others. 5. **Noise Level**: Opt for a printer with low noise levels (less than 40 dB) if you plan to use it in a quiet environment.  By considering these key features and best printer models, you'll be able to make an informed decision when choosing the right printer for your needs in 2024.
                Article #2:
                This question is a common query among individuals and businesses looking for reliable and high-quality printers that can meet their printing needs.  According to recent reviews and ratings, some of the top-rated printer models in 2024 include the HP OfficeJet Pro 9015, Epson Expression ET-2750 EcoTank, Canon imageCLASS LBP612Cdw, Brother MFC-J6930DW, Epson Expression Photo HD XP-15000: A high-end photo printer that offers excellent print quality, Wi-Fi connectivity, and a large 3.7-inch touchscreen display and Xerox VersaLink C405.  The HP OfficeJet Pro 9015 is a compact and feature-rich all-in-one printer that offers high-quality printing, scanning, and copying capabilities at an affordable price. It also features wireless connectivity, allowing users to print from their mobile devices.  On the other hand, the Epson Expression ET-2750 EcoTank is a budget-friendly option that offers superior ink tank capacity and ultra-low cost per page printing. This printer also comes with Wi-Fi connectivity and USB connectivity options for flexible use.  The Canon imageCLASS LBP612Cdw is a compact all-in-one laser printer that provides fast and reliable printing, scanning, and copying capabilities. It also features wireless connectivity and mobile printing capabilities.  Meanwhile, the Brother MFC-J6930DW is a versatile inkjet printer that offers high-quality printing, scanning, and copying capabilities at an affordable price. This printer comes with dual paper trays, allowing users to print on various media sizes without interruption. HP Envy 5055: A compact and affordable all-in-one inkjet printer that prints up to 9 ppm in black and white and 7 ppm in color. Lastly, the Xerox VersaLink C405 is a professional-grade laser printer designed for heavy-duty use in small businesses or workgroups. It features wireless connectivity, mobile printing, and cloud-based printing capabilities.  These top-rated printer models are available in various price ranges and come with different features and specifications to cater to diverse user needs. When choosing the best printer model in 2024, it is essential to consider factors such as print quality, speed, ink or toner cost, connectivity options, and compatibility with various devices.
                Article #3:
                This question can be answered by considering several factors such as budget, intended use, and features required. For home users, some top picks for printers include the HP Envy Pro 6455, Canon PIXMA TS6320, and Epson EcoTank ET-3750.  These printers offer a range of features including wireless connectivity, high-quality printing, and cost-effective ink options. The HP Envy Pro 6455 is an all-in-one printer that can print, scan, and copy documents with ease, making it suitable for home offices or small businesses.  The Canon PIXMA TS6320 offers excellent color accuracy and is ideal for those who want to print high-quality photos. It also has a compact design and wireless connectivity options. On the other hand, the Epson EcoTank ET-3750 is a great option for those on a budget as it comes with refillable ink tanks that can save users up to 90% of their printing costs.  For small businesses or professionals who need high-volume printing capabilities, some top picks include the HP LaserJet Pro M255dw, Brother HL-L2395DW, and Canon imageCLASS LBP612Cdw. These printers offer fast print speeds, high-quality text output, and wireless connectivity options to meet the demands of busy offices.  The HP LaserJet Pro M255dw is a monochrome laser printer that offers fast print speeds and wireless connectivity options. The Brother HL-L2395DW is another reliable option for small businesses or professionals who need high-volume printing capabilities. It comes with wireless connectivity options, automatic two-sided printing, and a compact design.  For photographers and graphic designers who require high-quality photo printing, some top picks include the Epson Expression Photo HD XP-15000, Canon PIXMA PRO-200, and HP DesignJet T830. These printers offer excellent color accuracy, high-quality photo printing capabilities, and wireless connectivity options to meet the demands of professionals.  The Epson Expression Photo HD XP-15000 is a great option for those who want to print high-quality photos with ease. It comes with refillable ink tanks that can save users up to 80% of their printing costs. Epson Expression Photo HD XP-15000: A high-end photo printer that offers excellent print quality, Wi-Fi connectivity, and a large 3.7-inch touchscreen display. The Canon PIXMA PRO-200 offers excellent color accuracy and is ideal for professional photographers or graphic designers.  In conclusion, the best printer models in 2024 depend on individual needs and requirements. Home users may prefer all-in-one printers like the HP Envy Pro 6455 or Epson EcoTank ET-3750, while small businesses or professionals may require high-volume printing capabilities from printers like the HP LaserJet Pro M255dw or Canon imageCLASS LBP612Cdw.  Photographers and graphic designers who require high-quality photo printing may prefer printers like the Epson Expression Photo HD XP-15000 or Canon PIXMA PRO-200. Xerox VersaLink C405 is a professional-grade laser printer designed for heavy-duty use in small businesses or workgroups. It features wireless connectivity, mobile printing, and cloud-based printing capabilities. Ultimately, choosing the best printer model depends on individual needs, budget, and intended use, making it essential to research and compare different options before making a final decision.  When selecting a printer in 2024, consider factors such as print quality, speed, connectivity options, and cost-effectiveness. Researching online reviews, comparing prices, and considering warranty and customer support can also help individuals make an informed decision when choosing the best printer model for their needs.  Furthermore, with advancements in technology, some printers now offer smart features like voice control, mobile printing capabilities, and automatic document feeders to streamline workflow and save time. Considering these factors will enable users to find a reliable and efficient printer that meets their specific requirements.  Lastly, it is worth noting that some manufacturers offer subscription-based services or cloud-connected features that can enhance the user experience and provide access to advanced features and benefits. These subscriptions may include ink replacement plans, automatic software updates, or enhanced security features, which can further optimize printing performance and convenience.  In summary, selecting the best printer model in 2024 requires careful consideration of individual needs, budget, and intended use. By researching different options, comparing prices, and considering factors such as print quality, speed, connectivity options, and cost-effectiveness, individuals can find a reliable and efficient printer that meets their specific requirements.  Additionally, exploring smart features, subscription-based services, and cloud-connected capabilities can further enhance the user experience and provide access to advanced features and benefits. With so many options available in 2024, there is sure to be a printer model that suits every need and budget, making it essential to do thorough research before making a final decision.  Overall, choosing the best printer model in 2024 requires considering various factors such as budget, intended use, print quality, speed, connectivity options, cost-effectiveness, and additional features like smart capabilities or subscription-based services.
                \
                Response: Here is the ranking list for the best printer models in 2024 with their summaries from most to least:
                \n
                **#1. Epson Expression Photo HD XP-15000:**
                'The Epson Expression Photo HD XP-15000 is a high-end photo printer that offers excellent print quality, Wi-Fi connectivity, and a large touchscreen display, making it an ideal option for those who want to print high-quality photos with ease. Additionally, the printer features refillable ink tanks that can save users up to 80% of their printing costs.'
                \n
                **#2. Xerox VersaLink C405:**
                'The Xerox VersaLink C405 is a powerful and versatile all-in-one printer that offers advanced security features, wireless connectivity, and a large paper capacity, making it suitable for heavy-duty use in small businesses or workgroups. Its professional-grade design enables mobile printing, cloud-based printing, and other convenient features that cater to the needs of busy office environments.'
                \n
                **#3. Canon PIXMA PRO-200:**
                'The Canon PIXMA PRO-200 is a professional-grade photo printer that delivers exceptional print quality while offering the convenience of wireless printing capabilities in a compact design.'
                \n
                **#4. HP Envy 5055:**
                'The HP Envy 5055 is a compact and affordable all-in-one inkjet printer that offers high print speeds, printing up to 9 pages per minute (ppm) in black and white and 7 ppm in color.'
                \n
                **#5. Canon PIXMA TS9521C:**
                'The Canon PIXMA TS9521C is a multifunctional printer that offers a range of features, including wireless printing capabilities, the ability to print on CDs and DVDs, and a sleek, compact design.'
                \n
                **#6. HP LaserJet Pro M283fdw:**
                'The HP LaserJet Pro M283fdw is a reliable and efficient monochrome laser printer capable of producing up to 30 pages per minute with a generous paper capacity.'
                \n
                **#7. Brother HL-L8360CDW:**
                'The Brother HL-L8360CDW is a high-performance color laser printer that offers wireless connectivity options through Wi-Fi, as well as near-field communication (NFC) capabilities, and features an intuitive touchscreen interface for easy operation.'
                \n
                **#8. Epson SureColor P800:**
                'The Brother HL-L8360CDW is a high-performance color laser printer that offers wireless connectivity options through Wi-Fi, as well as near-field communication (NFC) capabilities, and features an intuitive touchscreen interface for easy operation.'
                \n
                **#9. HP Envy 7855:**
                'The HP Envy 7855 is an all-in-one photo printer that offers versatility through its ability to print wirelessly, produce CDs and DVDs, and features a large 2.7-inch touchscreen display for user convenience.'
                \n
                **#10. Canon PIXMA TS202:**
                'The Canon PIXMA TS202 is a cost-effective all-in-one inkjet printer capable of printing at speeds of up to 8 pages per minute in black and white and 4 pages per minute in color.'
                \n
                **#11. Epson Expression Home XP-4100:**
                'The Epson Expression Home XP-4100 is a compact and budget-friendly all-in-one inkjet printer that offers wireless printing capabilities and features a user-friendly 2.7-inch touchscreen display.'
                \n
                **#12. HP DeskJet Ink Advantage 2135:**
                'The HP DeskJet Ink Advantage 2135 is a cost-effective, multifunctional inkjet printer that has the capability to print at speeds of up to 7 pages per minute in black and white and 4 pages per minute in color.'
                \n
                **#13. HP OfficeJet Pro 9015:**
                'The HP OfficeJet Pro 9015 is a compact all-in-one printer that provides high-quality printing, scanning, and copying abilities, as well as wireless connectivity for convenient mobile device printing at an affordable cost.'
                \n
                **#14. Epson Expression ET-2750 EcoTank:**
                'The Epson Expression ET-2750 EcoTank is a budget-friendly printer that provides superior ink tank capacity, allowing users to print at an extremely low cost per page, while also offering convenient connectivity options including Wi-Fi and USB.'
                \n
                **#15. Canon imageCLASS LBP612Cdw:**
                'The Canon imageCLASS LBP612Cdw is a compact all-in-one laser printer that offers fast and reliable performance, including printing, scanning, and copying functions, as well as wireless connectivity and mobile printing capabilities.'
                \n
                **#16. Brother MFC-J6930DW:**
                'The Brother MFC-J6930DW is a multifunctional inkjet printer that provides high-quality printing, scanning, and copying abilities at an affordable cost, also featuring dual paper trays for uninterrupted printing on various media sizes.'
                \n
                **#17. HP Envy Pro 6455:**
                'The HP Envy Pro 6455 is a versatile all-in-one printer designed to meet the needs of home offices or small businesses, offering printing, scanning, and copying capabilities.'
                \n
                **#18. Canon PIXMA TS6320:**
                'The Canon PIXMA TS6320 offers excellent color accuracy, making it suitable for printing high-quality photos, while its compact design and wireless connectivity features provide added convenience.'
                \n
                **#19. Epson EcoTank ET-3750:**
                'The Epson EcoTank ET-3750 is a cost-effective printer option, offering significant savings by using refillable ink tanks, which can reduce printing costs by up to 90%.'
                \n
                **#20. HP LaserJet Pro M255dw:**
                'The HP LaserJet Pro M255dw is a monochrome laser printer that provides quick print capabilities and allows for wireless connection options, making it convenient to use in various settings.'
                \n
                **#21. Brother HL-L2395DW:**
                'The Brother HL-L2395DW is a reliable printer option that is well-suited for small businesses or professionals who require high-volume printing capabilities, offering features such as wireless connectivity, automatic two-sided printing, and a compact design.'
                \n
                \n
                'When selecting the best printer for your needs in 2024, it is essential to consider several key features and factors. These include high print resolutions (at least 4800 x 1200 dpi) for excellent photo printing quality, wireless connectivity options such as Wi-Fi or Bluetooth, a large paper capacity to minimize refills and waste, affordable ink costs, and low noise levels for quiet environments. Additionally, individuals should research different models, compare prices, and consider factors such as print quality, speed, connectivity options, cost-effectiveness, and smart features like voice control and mobile printing capabilities. Furthermore, subscription-based services or cloud-connected features can provide access to advanced features and benefits, making it possible to find a reliable and efficient printer that meets specific requirements. By carefully considering individual needs, budget, and intended use, as well as exploring various options and features, individuals can make an informed decision when choosing the best printer model for 2024.'
                \
                ---\
                '''
        return few3_shot_prompt

In [20]:
def few3_shot_prompt_SINGLE(query):
        few3_shot_prompt =  f'''You are a reviewer generating an ordered list based on {query} only. \
                Each item in the ordered list should be ordered from item that appeared most amount of times in the user's text input to the item that appeared least amount of times in the user's text input. \
                Each item in the ordered list should have the item name and the item details from the user's text input. \
                Bold each item on the ranking list in the output.\
                To make text bold, enclose it with double asterisks (**). Example: **text** becomes bold.\
                ---\
                <EXAMPLE>\
                Prompt: In the context of "What are the top 5 best sci-fi movies of all time?" , generate an ordered list from the following text input: This is a question that has been debated by film enthusiasts for years, with different opinions and rankings depending on personal taste and preferences. However, based on critical acclaim, popularity, and impact on popular culture, here are five of the most highly-regarded science fiction films of all time.   Blade Runner (1982) directed by Ridley Scott is a groundbreaking sci-fi classic that has been widely acclaimed for its thought-provoking themes and visually stunning depiction of a dystopian future. The movie follows Rick Deckard, a bounty hunter tasked with tracking down advanced androids known as replicants who have escaped and are seeking to extend their short lifespan. Set in a polluted and overcrowded Los Angeles, the film explores complex questions about humanity, identity, and what it means to be alive.  Another iconic sci-fi film is The Matrix (1999) directed by the Wachowskis. This innovative movie revolutionized visual effects and special effects with its intricate "bullet time" sequences and stunning "green screen" work. The story revolves around Neo, a computer hacker who discovers that his entire life has been simulated within a complex virtual reality program created by intelligent machines to subdue humanity. As Neo delves deeper into the mysteries of this world, he must choose between his loyalty to humanity and his newfound connection with the machines.  Inception (2010) directed by Christopher Nolan is another thought-provoking sci-fi epic that has captivated audiences worldwide with its intricate plot and complex themes. The movie follows Cobb, a skilled thief who specializes in entering people\'s dreams and stealing their secrets. With the help of his team, Cobb embarks on a perilous mission to plant an idea in someone\'s mind instead of stealing it, but he soon finds himself trapped in a labyrinthine world where the boundaries between reality and fantasy blur.  Stanley Kubrick\'s 2001: A Space Odyssey (1968) is widely regarded as one of the greatest films of all time, and its sci-fi elements have had a profound influence on popular culture. The movie follows a team of astronauts on a mission to Jupiter, where they encounter a mysterious black monolith that holds the key to humanity\'s evolution and destiny in space. As the crew delves deeper into the mysteries of this monolith, they must confront their own existence and the possibility of extraterrestrial life.  The Last Jedi (2017) directed by Rian Johnson is often considered one of the greatest sci-fi movies of all time due to its bold storytelling, memorable characters, and subversive themes. The film takes place after the events of The Force Awakens and follows Rey as she embarks on a journey to find Luke Skywalker, who has gone into exile after his failure to stop Kylo Ren from destroying the Resistance\'s fleet. As Rey navigates this treacherous world, she must confront her own destiny, the true nature of the Force, and the secrets that have been hidden for centuries.  In conclusion, these five sci-fi movies are widely regarded as some of the greatest films ever made due to their thought-provoking themes, visually stunning depiction of a futuristic world, and profound impact on popular culture. Whether you\'re a fan of classic dystopian futures or cutting-edge space opera, these movies will transport you to new worlds and challenge your perceptions about humanity\'s place in the universe.  These five sci-fi films have stood the test of time due to their exceptional storytelling, memorable characters, and subversive themes that continue to captivate audiences worldwide. From Blade Runner\'s exploration of what it means to be human to The Matrix\'s innovative visual effects and complex world-building, each of these movies has contributed significantly to the sci-fi genre.
                This question has been debated by film enthusiasts and critics for decades, with various opinions on which films truly stand out as the greatest sci-fi movies ever made.  The Matrix (1999) directed by The Wachowskis is one of the most iconic sci-fi movies in recent history. It tells the story of Neo (Keanu Reeves), a computer hacker who discovers that his entire life has been a simulation created by intelligent machines to distract humans while their bodies are harvested for energy. With its innovative special effects, intricate storyline, and thought-provoking themes, The Matrix is widely regarded as one of the greatest sci-fi films ever made.  Blade Runner (1982) directed by Ridley Scott is another classic sci-fi movie that explores what it means to be human. Set in a dystopian future where synthetic beings known as replicants are hunted by special police officers called blade runners, the film delves into complex questions about artificial intelligence, empathy, and humanity. With its atmospheric visuals, rich themes, and memorable characters, Blade Runner has become a cult classic among sci-fi fans.  2001: A Space Odyssey (1968) directed by Stanley Kubrick is often cited as one of the most influential films in science fiction history. This groundbreaking film tells the story of a team of astronauts who embark on a mission to Jupiter, where they encounter a mysterious alien monolith that holds the key to humanity's evolution. With its groundbreaking special effects, philosophical themes, and enigmatic ending, it continues to inspire new generations of filmmakers and sci-fi enthusiasts.  Alien (1979) directed by Ridley Scott is another iconic sci-fi movie that has become a staple of the genre. The film tells the story of a crew of space explorers who are stalked and killed one by one by a deadly alien creature on a distant planet. With its claustrophobic tension, memorable characters, and groundbreaking visual effects, Alien has become one of the most beloved sci-fi horror films of all time.  In conclusion, the top 5 best sci-fi movies of all time include The Matrix (1999), Blade Runner (1982), 2001: A Space Odyssey (1968), Alien (1979), and Star Wars: Episode IV - A New Hope (1977). These films have captivated audiences with their imaginative storylines, memorable characters, and groundbreaking special effects, solidifying their places in the pantheon of science fiction cinema.
                This question is often subjective, as opinions may vary depending on personal taste and preferences. However, here is a summary of some of the most iconic and influential science fiction films that have made it to various "best of" lists:  One of the most popular and enduring sci-fi movies of all time is Stanley Kubrick\'s "2001: A Space Odyssey" (1968). Directed by Kubrick and written by Arthur C. Clarke, this groundbreaking film follows a team of astronauts on a mission to Jupiter and explores themes of human evolution, technology, and existentialism. The movie\'s slow pace and philosophical undertones were considered revolutionary at the time of its release.  Another classic sci-fi film that has stood the test of time is Ridley Scott\'s "Blade Runner" (1982). Set in a dystopian future where synthetic humans are hunted by special police officers known as blade runners, this visually stunning movie explores complex themes such as humanity, artificial intelligence, and what it means to be alive. The film\'s atmospheric and introspective tone has influenced countless other sci-fi movies and TV shows.  Martin Scorsese\'s "Alien" (1979) is a seminal work of science fiction that revolutionized the horror genre with its terrifying alien creatures and claustrophobic setting. Directed by Ridley Scott, this movie follows a crew of space explorers who are stalked and killed by a deadly extraterrestrial entity, leading to a thrilling cat-and-mouse chase through the spaceship\'s corridors.  Steven Spielberg\'s "E.T. the Extra-Terrestrial" (1982) is often cited as one of the greatest family movies of all time, but it also explores complex themes such as first contact, xenophobia, and the power of childhood innocence. The movie follows a young boy named Elliott who befriends an alien who has been left behind on Earth and must help him return home.  The Matrix (1999) is another influential sci-fi movie that redefined the genre with its innovative special effects, intricate storyline, and thought-provoking themes. Directed by the Wachowskis, this movie follows a computer hacker named Neo who discovers that his entire life has been a simulated reality created by sentient machines to distract humans while their bodies are harvested for energy.  These five movies have all had a significant impact on popular culture and continue to be widely discussed and debated by fans of science fiction. While opinions may vary, they are generally regarded as some of the greatest sci-fi films of all time due to their thought-provoking themes, visually stunning cinematography, and enduring influence on the genre.  In conclusion, these five movies have captured the imagination of audiences worldwide with their complex characters, engaging storylines, and exploration of deeper themes. They continue to be celebrated as some of the greatest sci-fi films of all time and remain essential viewing for fans of science fiction and anyone interested in exploring new ideas and perspectives.
                \
                Response: Here is the ranking list for the top 5 best sci-fi movies of all time with their summaries from most to least:
                \n
                **#1. Blade Runner:**
                'Blade Runner, directed by Ridley Scott in 1982, is a groundbreaking and thought-provoking sci-fi classic that explores complex themes about humanity, identity, and what it means to be alive. Set in a dystopian future where synthetic beings known as replicants are hunted by special police officers called blade runners, the film delves into questions about artificial intelligence, empathy, and humanity through its atmospheric visuals, rich themes, and memorable characters, ultimately becoming a cult classic among sci-fi fans that has influenced numerous other movies and TV shows.'
                \n
                **#2. The Matrix: **
                'The Matrix (1999), directed by the Wachowskis, is an iconic and influential sci-fi movie that revolutionized visual effects with its "bullet time" sequences and "green screen" work. The film tells the story of Neo, a computer hacker who discovers his life has been simulated within a virtual reality program created by intelligent machines to subdue humanity, forcing him to choose between loyalty to humans and connection with the machines. With its innovative special effects, intricate storyline, and thought-provoking themes, The Matrix is widely regarded as one of the greatest sci-fi films ever made, redefining the genre with its groundbreaking storytelling and visuals.'
                \n
                **#3. 2001: A Space Odyssey: **
                "Stanley Kubrick's 2001: A Space Odyssey (1968) is widely regarded as one of the greatest films of all time and has had a profound influence on popular culture, particularly in the science fiction genre. The film follows a team of astronauts on a mission to Jupiter who encounter a mysterious alien monolith that holds the key to humanity's evolution and destiny, prompting them to confront their own existence and the possibility of extraterrestrial life. With its groundbreaking special effects, philosophical themes, and enigmatic ending, 2001: A Space Odyssey continues to inspire new generations of filmmakers and sci-fi enthusiasts with its thought-provoking exploration of human evolution, technology, and existentialism."
                \n
                **#4. Alien: **
                'The iconic sci-fi horror film Alien, directed by Ridley Scott in 1979, tells the story of a crew of space explorers being stalked and killed one by one by a deadly alien creature on a distant planet. With its groundbreaking visual effects, memorable characters, and claustrophobic tension, Alien has become a seminal work of science fiction that revolutionized the horror genre with its terrifying alien creatures and eerie setting.'
                \n
                **#5. The Last Jedi: **
                "The Last Jedi, directed by Rian Johnson, is often considered one of the greatest sci-fi movies of all time due to its bold storytelling, memorable characters, and subversive themes. The film follows Rey's journey as she searches for Luke Skywalker, who has gone into hiding after his failure to stop Kylo Ren from destroying the Resistance's fleet, while also grappling with her own destiny and the mysteries of the Force."
                \n
                \n
                'The five sci-fi movies discussed are widely regarded as some of the greatest films ever made due to their thought-provoking themes, visually stunning depiction of a futuristic world, and profound impact on popular culture. These movies have captivated audiences with their imaginative storylines, memorable characters, and groundbreaking special effects, solidifying their places in the pantheon of science fiction cinema and continuing to be widely discussed and debated by fans of science fiction. Their complex world-building, exploration of deeper themes, and enduring influence on the genre have cemented their status as essential viewing for fans of science fiction and anyone interested in exploring new ideas and perspectives.'
                \
                \
                ---\
                <EXAMPLE>\
                Prompt: In the context of "What are the 3 worst games consoles of all time?" , generate an ordered list from the following text input: The Atari Jaguar is often cited as one of the most disappointing consoles ever made, with its high price and lackluster lineup of games failing to attract a significant audience.  The Sega Dreamcast was another console that failed to gain traction in the market, despite its innovative controller design and online capabilities. Its short lifespan and limited support from third-party developers also contributed to its failure. The console is still remembered fondly by some enthusiasts for its quirky charm and exclusive games such as Sonic Adventure and Crazy Taxi.  The Nintendo Virtual Boy is widely regarded as one of the worst consoles ever made, with its poor graphics, lackluster lineup of games, and high price point making it a flop in the market. The console's failure was so severe that it led to a significant decline in Nintendo's stock value and a reevaluation of their hardware design strategies.  The Atari 7800 is another console often cited as one of the worst due to its poor design and lackluster lineup of games, with many critics feeling that it was little more than an improved version of the Atari 2600. The console's high price point and limited support from third-party developers also contributed to its failure in the market.  The Sega Nomad is often cited as one of the worst handheld consoles ever made due to its poor battery life, lackluster lineup of games, and high price point. Its design flaws and limited functionality made it unpopular among gamers, leading to a significant decline in sales for Sega.  The Nintendo 64DD was another console that failed to gain traction in the market, with its slow loading times, limited support from third-party developers, and lackluster lineup of games making it unpopular among gamers. Its short lifespan and poor marketing also contributed to its failure, ultimately leading to a loss for Nintendo.  The Sega SG-1000 is often cited as one of the worst consoles ever made due to its poor design, lackluster lineup of games, and high price point. Its limited functionality and compatibility issues made it unpopular among gamers, leading to a significant decline in sales for Sega.  Each of these consoles has been remembered fondly by some enthusiasts for their quirks and charm, but overall they are widely regarded as some of the worst consoles ever made due to their poor design, lackluster lineups of games, and limited functionality.
                When evaluating the worst gaming consoles ever made, it's essential to consider several factors such as their technical issues, poor game library, and overall impact on the gaming industry.  One of the worst consoles is the Sega Dreamcast released in 1998. Although it received widespread critical acclaim for its innovative design and impressive games like Sonic Adventure and Crazy Taxi, it struggled financially due to a lack of third-party support and limited release in various countries. This led to a significant decline in sales, ultimately causing the console's demise.  Another contender is the Nintendo Virtual Boy released in 1995. Although touted as the successor to the Super Nintendo, this console failed miserably due to its poor game library and numerous technical issues. Its games were often criticized for their low resolution and lackluster graphics, which further exacerbated the console's failure.  Lastly, the Atari Jaguar, released in 1993, is also considered one of the worst consoles ever made. Despite being launched with significant hype, it suffered from a limited game library and numerous technical issues such as overheating and poor sound quality. Its failure was so catastrophic that it nearly bankrupted the company, leading to a massive decline in its market value.  The three consoles were unable to gain substantial market share or even stay afloat for an extended period. While some may argue that their failures led to improvements in future gaming technology and innovations, they undoubtedly remain among the worst console releases in history.
                The Atari Jaguar, Sega Saturn, and Nintendo Virtual Boy have been widely regarded as some of the worst game consoles ever released.  The Atari Jaguar was released in 1993 and failed miserably due to its high price tag, poor marketing, and lackluster library of games. It had a short lifespan, lasting only three years before being discontinued. The console's failure was attributed to its inability to compete with other popular consoles such as the Sega Genesis and Super Nintendo Entertainment System.  The Sega Saturn, released in 1994, was plagued by poor marketing and high development costs. Despite its impressive graphics capabilities, it failed to gain significant market share due to its limited game library and lack of third-party support. The console's failure led to a significant decline in Sega's fortunes, resulting in the company eventually abandoning the Saturn and shifting focus to other ventures.  The Nintendo Virtual Boy was released in 1995 as a pioneering attempt at virtual reality gaming but ultimately failed due to its high price, lackluster library of games, and limited lifespan. The console was criticized for its poor graphics, awkward design, and lack of appeal to mainstream gamers. Its failure led to Nintendo abandoning the concept of virtual reality gaming for several years.  In conclusion, these three consoles are often cited as some of the worst game consoles ever released due to their high prices, poor marketing, and limited libraries of games.
                \
                Response: Here is the ranking list for the 3 worst games consoles of all time with their summaries from the most to least bad:
                \n
                **#1. Atari Jaguar:**
                The Atari Jaguar is widely regarded as one of the most disappointing consoles ever made, released in 1993 with high hopes but ultimately failing due to a combination of factors including its high price tag, limited game library, and numerous technical issues. Its catastrophic failure nearly bankrupted the company and led to a significant decline in market value, forcing the console's discontinuation just three years after its release.
                \n
                **#2. Nintendo Virtual Boy:**
                The Nintendo Virtual Boy is widely regarded as one of the worst consoles ever made due to its poor graphics, limited game library, and high price point, which ultimately led to a significant decline in Nintendo's stock value and a reevaluation of their hardware design strategies. Released in 1995 as an attempt at virtual reality gaming, the console failed miserably due to its numerous technical issues, awkward design, and lack of appeal to mainstream gamers, causing Nintendo to abandon the concept of virtual reality gaming for several years.
                \n
                **#3. Sega Dreamcast:**
                The Sega Dreamcast was a console that failed to gain significant market traction, despite its innovative controller design and online capabilities, due to its short lifespan and limited third-party developer support. Despite receiving critical acclaim for its games such as Sonic Adventure and Crazy Taxi, it struggled financially and ultimately declined in sales, leading to its demise.
                \n
                The three consoles described in the text have been widely regarded as among the worst console releases in history due to their poor design, lackluster lineups of games, and limited functionality. Despite some potential benefits from their failures that led to innovations in future gaming technology, these consoles were unable to gain substantial market share or stay afloat for an extended period, ultimately being remembered more for their quirks and flaws than any redeeming qualities.
                \
                \
                ---\
                <EXAMPLE>\
                Prompt: In the context of "What are the best printer models in 2024?" , generate an ordered list from the following text input: Choosing the right printer can be overwhelming with so many options available, but here's a comprehensive guide to help you make an informed decision.  **Best Printer Models for Home Users**  1. **HP Envy 5055**: A compact and affordable all-in-one inkjet printer that prints up to 9 ppm in black and white and 7 ppm in color. 2. **Epson Expression Photo HD XP-15000**: A high-end photo printer that offers excellent print quality, Wi-Fi connectivity, and a large 3.7-inch touchscreen display. 3. **Canon PIXMA TS9521C**: A versatile all-in-one printer with wireless printing capabilities, CD/DVD printing, and a compact design.  **Best Printer Models for Office Users**  1. **HP LaserJet Pro M283fdw**: A reliable and high-speed monochrome laser printer that prints up to 30 ppm, has a large paper capacity, and supports mobile printing. 2. **Brother HL-L8360CDW**: A fast and feature-rich color laser printer with Wi-Fi connectivity, NFC, and a touchscreen display. 3. **Xerox VersaLink C405**: A powerful and versatile all-in-one printer that offers advanced security features, wireless connectivity, and a large paper capacity.  **Best Printer Models for Photo Printing**  1. **Epson SureColor P800**: A high-end photo printer with excellent print quality, Wi-Fi connectivity, and a range of paper sizes. 2. **Canon PIXMA PRO-200**: A professional-grade photo printer that offers exceptional print quality, wireless printing capabilities, and a compact design. 3. **HP Envy 7855**: A versatile all-in-one photo printer with wireless printing capabilities, CD/DVD printing, and a large 2.7-inch touchscreen display.  **Best Printer Models for Budget-Friendly Options**  1. **Canon PIXMA TS202**: A budget-friendly all-in-one inkjet printer that prints up to 8 ppm in black and white and 4 ppm in color. 2. **Epson Expression Home XP-4100**: A compact and affordable all-in-one inkjet printer with wireless printing capabilities and a large 2.7-inch touchscreen display. 3. **HP DeskJet Ink Advantage 2135**: A budget-friendly all-in-one inkjet printer that prints up to 7 ppm in black and white and 4 ppm in color.  **Key Features to Consider**  1. **Print Resolution**: Look for printers with high print resolutions (at least 4800 x 1200 dpi) for excellent photo printing quality. 2. **Wireless Connectivity**: Choose a printer with Wi-Fi or Bluetooth connectivity for convenient wireless printing from mobile devices. 3. **Paper Capacity**: Select a printer with a large paper capacity to minimize refills and reduce waste. 4. **Ink Costs**: Consider the cost of replacement ink cartridges when choosing a printer, as some models may be more expensive than others. 5. **Noise Level**: Opt for a printer with low noise levels (less than 40 dB) if you plan to use it in a quiet environment.  By considering these key features and best printer models, you'll be able to make an informed decision when choosing the right printer for your needs in 2024.
                This question is a common query among individuals and businesses looking for reliable and high-quality printers that can meet their printing needs.  According to recent reviews and ratings, some of the top-rated printer models in 2024 include the HP OfficeJet Pro 9015, Epson Expression ET-2750 EcoTank, Canon imageCLASS LBP612Cdw, Brother MFC-J6930DW, Epson Expression Photo HD XP-15000: A high-end photo printer that offers excellent print quality, Wi-Fi connectivity, and a large 3.7-inch touchscreen display and Xerox VersaLink C405.  The HP OfficeJet Pro 9015 is a compact and feature-rich all-in-one printer that offers high-quality printing, scanning, and copying capabilities at an affordable price. It also features wireless connectivity, allowing users to print from their mobile devices.  On the other hand, the Epson Expression ET-2750 EcoTank is a budget-friendly option that offers superior ink tank capacity and ultra-low cost per page printing. This printer also comes with Wi-Fi connectivity and USB connectivity options for flexible use.  The Canon imageCLASS LBP612Cdw is a compact all-in-one laser printer that provides fast and reliable printing, scanning, and copying capabilities. It also features wireless connectivity and mobile printing capabilities.  Meanwhile, the Brother MFC-J6930DW is a versatile inkjet printer that offers high-quality printing, scanning, and copying capabilities at an affordable price. This printer comes with dual paper trays, allowing users to print on various media sizes without interruption. HP Envy 5055: A compact and affordable all-in-one inkjet printer that prints up to 9 ppm in black and white and 7 ppm in color. Lastly, the Xerox VersaLink C405 is a professional-grade laser printer designed for heavy-duty use in small businesses or workgroups. It features wireless connectivity, mobile printing, and cloud-based printing capabilities.  These top-rated printer models are available in various price ranges and come with different features and specifications to cater to diverse user needs. When choosing the best printer model in 2024, it is essential to consider factors such as print quality, speed, ink or toner cost, connectivity options, and compatibility with various devices.
                This question can be answered by considering several factors such as budget, intended use, and features required. For home users, some top picks for printers include the HP Envy Pro 6455, Canon PIXMA TS6320, and Epson EcoTank ET-3750.  These printers offer a range of features including wireless connectivity, high-quality printing, and cost-effective ink options. The HP Envy Pro 6455 is an all-in-one printer that can print, scan, and copy documents with ease, making it suitable for home offices or small businesses.  The Canon PIXMA TS6320 offers excellent color accuracy and is ideal for those who want to print high-quality photos. It also has a compact design and wireless connectivity options. On the other hand, the Epson EcoTank ET-3750 is a great option for those on a budget as it comes with refillable ink tanks that can save users up to 90% of their printing costs.  For small businesses or professionals who need high-volume printing capabilities, some top picks include the HP LaserJet Pro M255dw, Brother HL-L2395DW, and Canon imageCLASS LBP612Cdw. These printers offer fast print speeds, high-quality text output, and wireless connectivity options to meet the demands of busy offices.  The HP LaserJet Pro M255dw is a monochrome laser printer that offers fast print speeds and wireless connectivity options. The Brother HL-L2395DW is another reliable option for small businesses or professionals who need high-volume printing capabilities. It comes with wireless connectivity options, automatic two-sided printing, and a compact design.  For photographers and graphic designers who require high-quality photo printing, some top picks include the Epson Expression Photo HD XP-15000, Canon PIXMA PRO-200, and HP DesignJet T830. These printers offer excellent color accuracy, high-quality photo printing capabilities, and wireless connectivity options to meet the demands of professionals.  The Epson Expression Photo HD XP-15000 is a great option for those who want to print high-quality photos with ease. It comes with refillable ink tanks that can save users up to 80% of their printing costs. Epson Expression Photo HD XP-15000: A high-end photo printer that offers excellent print quality, Wi-Fi connectivity, and a large 3.7-inch touchscreen display. The Canon PIXMA PRO-200 offers excellent color accuracy and is ideal for professional photographers or graphic designers.  In conclusion, the best printer models in 2024 depend on individual needs and requirements. Home users may prefer all-in-one printers like the HP Envy Pro 6455 or Epson EcoTank ET-3750, while small businesses or professionals may require high-volume printing capabilities from printers like the HP LaserJet Pro M255dw or Canon imageCLASS LBP612Cdw.  Photographers and graphic designers who require high-quality photo printing may prefer printers like the Epson Expression Photo HD XP-15000 or Canon PIXMA PRO-200. Xerox VersaLink C405 is a professional-grade laser printer designed for heavy-duty use in small businesses or workgroups. It features wireless connectivity, mobile printing, and cloud-based printing capabilities. Ultimately, choosing the best printer model depends on individual needs, budget, and intended use, making it essential to research and compare different options before making a final decision.  When selecting a printer in 2024, consider factors such as print quality, speed, connectivity options, and cost-effectiveness. Researching online reviews, comparing prices, and considering warranty and customer support can also help individuals make an informed decision when choosing the best printer model for their needs.  Furthermore, with advancements in technology, some printers now offer smart features like voice control, mobile printing capabilities, and automatic document feeders to streamline workflow and save time. Considering these factors will enable users to find a reliable and efficient printer that meets their specific requirements.  Lastly, it is worth noting that some manufacturers offer subscription-based services or cloud-connected features that can enhance the user experience and provide access to advanced features and benefits. These subscriptions may include ink replacement plans, automatic software updates, or enhanced security features, which can further optimize printing performance and convenience.  In summary, selecting the best printer model in 2024 requires careful consideration of individual needs, budget, and intended use. By researching different options, comparing prices, and considering factors such as print quality, speed, connectivity options, and cost-effectiveness, individuals can find a reliable and efficient printer that meets their specific requirements.  Additionally, exploring smart features, subscription-based services, and cloud-connected capabilities can further enhance the user experience and provide access to advanced features and benefits. With so many options available in 2024, there is sure to be a printer model that suits every need and budget, making it essential to do thorough research before making a final decision.  Overall, choosing the best printer model in 2024 requires considering various factors such as budget, intended use, print quality, speed, connectivity options, cost-effectiveness, and additional features like smart capabilities or subscription-based services.
                \
                Response: Here is the ranking list for the best printer models in 2024 with their summaries from most to least:
                \n
                **#1. Epson Expression Photo HD XP-15000:**
                'The Epson Expression Photo HD XP-15000 is a high-end photo printer that offers excellent print quality, Wi-Fi connectivity, and a large touchscreen display, making it an ideal option for those who want to print high-quality photos with ease. Additionally, the printer features refillable ink tanks that can save users up to 80% of their printing costs.'
                \n
                **#2. Xerox VersaLink C405:**
                'The Xerox VersaLink C405 is a powerful and versatile all-in-one printer that offers advanced security features, wireless connectivity, and a large paper capacity, making it suitable for heavy-duty use in small businesses or workgroups. Its professional-grade design enables mobile printing, cloud-based printing, and other convenient features that cater to the needs of busy office environments.'
                \n
                **#3. Canon PIXMA PRO-200:**
                'The Canon PIXMA PRO-200 is a professional-grade photo printer that delivers exceptional print quality while offering the convenience of wireless printing capabilities in a compact design.'
                \n
                **#4. HP Envy 5055:**
                'The HP Envy 5055 is a compact and affordable all-in-one inkjet printer that offers high print speeds, printing up to 9 pages per minute (ppm) in black and white and 7 ppm in color.'
                \n
                **#5. Canon PIXMA TS9521C:**
                'The Canon PIXMA TS9521C is a multifunctional printer that offers a range of features, including wireless printing capabilities, the ability to print on CDs and DVDs, and a sleek, compact design.'
                \n
                **#6. HP LaserJet Pro M283fdw:**
                'The HP LaserJet Pro M283fdw is a reliable and efficient monochrome laser printer capable of producing up to 30 pages per minute with a generous paper capacity.'
                \n
                **#7. Brother HL-L8360CDW:**
                'The Brother HL-L8360CDW is a high-performance color laser printer that offers wireless connectivity options through Wi-Fi, as well as near-field communication (NFC) capabilities, and features an intuitive touchscreen interface for easy operation.'
                \n
                **#8. Epson SureColor P800:**
                'The Brother HL-L8360CDW is a high-performance color laser printer that offers wireless connectivity options through Wi-Fi, as well as near-field communication (NFC) capabilities, and features an intuitive touchscreen interface for easy operation.'
                \n
                **#9. HP Envy 7855:**
                'The HP Envy 7855 is an all-in-one photo printer that offers versatility through its ability to print wirelessly, produce CDs and DVDs, and features a large 2.7-inch touchscreen display for user convenience.'
                \n
                **#10. Canon PIXMA TS202:**
                'The Canon PIXMA TS202 is a cost-effective all-in-one inkjet printer capable of printing at speeds of up to 8 pages per minute in black and white and 4 pages per minute in color.'
                \n
                **#11. Epson Expression Home XP-4100:**
                'The Epson Expression Home XP-4100 is a compact and budget-friendly all-in-one inkjet printer that offers wireless printing capabilities and features a user-friendly 2.7-inch touchscreen display.'
                \n
                **#12. HP DeskJet Ink Advantage 2135:**
                'The HP DeskJet Ink Advantage 2135 is a cost-effective, multifunctional inkjet printer that has the capability to print at speeds of up to 7 pages per minute in black and white and 4 pages per minute in color.'
                \n
                **#13. HP OfficeJet Pro 9015:**
                'The HP OfficeJet Pro 9015 is a compact all-in-one printer that provides high-quality printing, scanning, and copying abilities, as well as wireless connectivity for convenient mobile device printing at an affordable cost.'
                \n
                **#14. Epson Expression ET-2750 EcoTank:**
                'The Epson Expression ET-2750 EcoTank is a budget-friendly printer that provides superior ink tank capacity, allowing users to print at an extremely low cost per page, while also offering convenient connectivity options including Wi-Fi and USB.'
                \n
                **#15. Canon imageCLASS LBP612Cdw:**
                'The Canon imageCLASS LBP612Cdw is a compact all-in-one laser printer that offers fast and reliable performance, including printing, scanning, and copying functions, as well as wireless connectivity and mobile printing capabilities.'
                \n
                **#16. Brother MFC-J6930DW:**
                'The Brother MFC-J6930DW is a multifunctional inkjet printer that provides high-quality printing, scanning, and copying abilities at an affordable cost, also featuring dual paper trays for uninterrupted printing on various media sizes.'
                \n
                **#17. HP Envy Pro 6455:**
                'The HP Envy Pro 6455 is a versatile all-in-one printer designed to meet the needs of home offices or small businesses, offering printing, scanning, and copying capabilities.'
                \n
                **#18. Canon PIXMA TS6320:**
                'The Canon PIXMA TS6320 offers excellent color accuracy, making it suitable for printing high-quality photos, while its compact design and wireless connectivity features provide added convenience.'
                \n
                **#19. Epson EcoTank ET-3750:**
                'The Epson EcoTank ET-3750 is a cost-effective printer option, offering significant savings by using refillable ink tanks, which can reduce printing costs by up to 90%.'
                \n
                **#20. HP LaserJet Pro M255dw:**
                'The HP LaserJet Pro M255dw is a monochrome laser printer that provides quick print capabilities and allows for wireless connection options, making it convenient to use in various settings.'
                \n
                **#21. Brother HL-L2395DW:**
                'The Brother HL-L2395DW is a reliable printer option that is well-suited for small businesses or professionals who require high-volume printing capabilities, offering features such as wireless connectivity, automatic two-sided printing, and a compact design.'
                \n
                \n
                'When selecting the best printer for your needs in 2024, it is essential to consider several key features and factors. These include high print resolutions (at least 4800 x 1200 dpi) for excellent photo printing quality, wireless connectivity options such as Wi-Fi or Bluetooth, a large paper capacity to minimize refills and waste, affordable ink costs, and low noise levels for quiet environments. Additionally, individuals should research different models, compare prices, and consider factors such as print quality, speed, connectivity options, cost-effectiveness, and smart features like voice control and mobile printing capabilities. Furthermore, subscription-based services or cloud-connected features can provide access to advanced features and benefits, making it possible to find a reliable and efficient printer that meets specific requirements. By carefully considering individual needs, budget, and intended use, as well as exploring various options and features, individuals can make an informed decision when choosing the best printer model for 2024.'
                \
                ---\
                '''
        return few3_shot_prompt

### Response Generation for Zero Shot prompting with SLT input

In [24]:
zero_shot_multi_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','avg_score'])

for index ,row in snippet_prompt_df.iterrows():
    text = row.Modified_Multiple_Generated_Summaries
    query = row.Query
    prompt = zero_shot_prompt_MULTI(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    avg_score = ''
    
    response_text_1 = MULTI_generate_report_with_query(text, model,query,prompt)
    response_text_2 = MULTI_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               avg_score,
               ]
    
    zero_shot_multi_response_df.loc[-1] = ref_row
    zero_shot_multi_response_df.index = zero_shot_multi_response_df.index + 1  #shift index
    zero_shot_multi_response_df = zero_shot_multi_response_df.sort_index()  #sort by index

In [25]:
zero_shot_multi_response_df.to_csv('zero_shot_multi_response_df_part_2.csv')

In [ ]:
zero_shot_multi_response_df.to_csv('zero_shot_SLT_response_SecondPart.csv')

### Response Generation for Zero Shot prompting with SCT input

In [26]:
zero_shot_single_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','avg_score'])

for index ,row in snippet_prompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    avg_score = ''
    
    response_text_1 = SINGLE_generate_report_with_query(text, model,query,prompt)
    response_text_2 = SINGLE_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               avg_score,
               ]
    
    zero_shot_single_response_df.loc[-1] = ref_row
    zero_shot_single_response_df.index = zero_shot_single_response_df.index + 1  #shift index
    zero_shot_single_response_df = zero_shot_single_response_df.sort_index()  #sort by index

In [27]:
zero_shot_single_response_df.to_csv('zero_shot_single_response_df_part_2.csv')

In [ ]:
zero_shot_single_response_df.to_csv('zero_shot_SCT_response_SecondPart.csv')

### Response Generation for Few (3) Shot prompting with SLT input

In [28]:
few3_shot_multi_response_df = pd.DataFrame(columns=['query','model_response','few3_shot_response_1','few3_shot_response_1_score','few3_shot_response_2','few3_shot_response_2_score','avg_score'])

for index ,row in snippet_prompt_df.iterrows():
    text = row.Modified_Multiple_Generated_Summaries
    query = row.Query
    prompt = few3_shot_prompt_MULTI(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    avg_score = ''
    
    response_text_1 = MULTI_generate_report_with_query(text, model,query,prompt)
    response_text_2 = MULTI_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               avg_score,
               ]
    
    few3_shot_multi_response_df.loc[-1] = ref_row
    few3_shot_multi_response_df.index = few3_shot_multi_response_df.index + 1  #shift index
    few3_shot_multi_response_df = few3_shot_multi_response_df.sort_index()  #sort by index

In [29]:
few3_shot_multi_response_df.to_csv('few3_shot_multi_response_df_part_2.csv')

In [ ]:
few3_shot_multi_response_df.to_csv('few3_shot_SLT_response_SecondPart.csv')

### Response Generation for Few (3) Shot prompting with SCT input

In [30]:
few3_shot_single_response_df = pd.DataFrame(columns=['query','model_response','few3_shot_response_1','few3_shot_response_1_score','few3_shot_response_2','few3_shot_response_2_score','avg_score'])

for index ,row in snippet_prompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = few3_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    avg_score = ''
    
    response_text_1 = SINGLE_generate_report_with_query(text, model,query,prompt)
    response_text_2 = SINGLE_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               avg_score,
               ]
    
    few3_shot_single_response_df.loc[-1] = ref_row
    few3_shot_single_response_df.index = few3_shot_single_response_df.index + 1  #shift index
    few3_shot_single_response_df = few3_shot_single_response_df.sort_index()  #sort by index

In [31]:
few3_shot_single_response_df.to_csv('few3_shot_single_response_df_part_2.csv')

In [ ]:
few3_shot_single_response_df.to_csv('few3_shot_SCT_response_SecondPart.csv')

### Metrics Evaluation

In [32]:
# Response Dataframes from previous sections

response_df_list= [zero_shot_multi_response_df,zero_shot_single_response_df,few3_shot_multi_response_df,few3_shot_single_response_df]
response_df_list_names =['zero_shot_multi_response_df','zero_shot_single_response_df','few3_shot_multi_response_df','few3_shot_single_response_df']

#### Getting ROUGE Scores

In [35]:
## Rouge Scores
AllRouge_Scores = pd.DataFrame()
AllRouge_Scores = pd.DataFrame(columns=['Prompt Engineering Technique','rouge-1-r','rouge-1-p','rouge-1-f','rouge-2-r','rouge-2-p','rouge-2-f','rouge-l-r','rouge-l-p','rouge-l-f'])

# llm_model = ['phi3:latest','llama3.1:latest']
for response_df_index in range(len(response_df_list)): 
    Response_cols = list(filter(lambda x: (('score' not in x) and ('shot_response' in x)) == True, list(response_df_list[response_df_index].columns)))
    print('Response_cols : ',Response_cols)
    ModelSummary_col = 'model_response'
    technique_name = response_df_list_names[response_df_index]
    print('technique_name : ',technique_name)
    
    
    response_df_counter=1
    for Response_col in Response_cols:
        test_rouge_df = response_df_list[response_df_index][[Response_col,ModelSummary_col]].copy(deep=True)
        test_rouge_df[Response_col] = test_rouge_df[Response_col].replace('', np.nan)
        test_rouge_df = test_rouge_df.dropna(subset=[Response_col])
        test_rouge_df = test_rouge_df.reset_index(drop=True)
        
        if response_df_counter ==1:
            rouge_scores_1 = eval_metrics.get_rouge_scores(test_rouge_df[ModelSummary_col],test_rouge_df[Response_col])
        else:
            rouge_scores_2 = eval_metrics.get_rouge_scores(test_rouge_df[ModelSummary_col],test_rouge_df[Response_col])
        response_df_counter+=1
    
    print('rouge_scores_1 : ',rouge_scores_1)
    print('rouge_scores_2 : ',rouge_scores_2)
    
    ref_row = [technique_name , 
               (rouge_scores_1['rouge-1']['r'] + rouge_scores_2['rouge-1']['r'])/2, 
               (rouge_scores_1['rouge-1']['p'] + rouge_scores_2['rouge-1']['p'])/2, 
               (rouge_scores_1['rouge-1']['f'] + rouge_scores_2['rouge-1']['f'])/2, 
               (rouge_scores_1['rouge-2']['r'] + rouge_scores_2['rouge-2']['r'])/2, 
               (rouge_scores_1['rouge-2']['p'] + rouge_scores_2['rouge-2']['p'])/2, 
               (rouge_scores_1['rouge-2']['f'] + rouge_scores_2['rouge-2']['f'])/2,
               (rouge_scores_1['rouge-l']['r'] + rouge_scores_2['rouge-l']['r'])/2, 
               (rouge_scores_1['rouge-l']['p'] + rouge_scores_2['rouge-l']['p'])/2, 
               (rouge_scores_1['rouge-l']['f'] + rouge_scores_2['rouge-l']['f'])/2
               ]
    
    print('ref_row : ',ref_row)
    
    AllRouge_Scores.loc[-1] = ref_row
    AllRouge_Scores.index = AllRouge_Scores.index + 1  #shift index
    AllRouge_Scores = AllRouge_Scores.sort_index()  #sort by index
    

Response_cols :  ['zero_shot_response_1', 'zero_shot_response_2']
technique_name :  zero_shot_multi_response_df
rouge_scores_1 :  {'rouge-1': {'r': 0.33964458427963573, 'p': 0.5698947325463974, 'f': 0.4203347210713083}, 'rouge-2': {'r': 0.16374614046043084, 'p': 0.313141412641913, 'f': 0.21116796061755724}, 'rouge-l': {'r': 0.31629588303810824, 'p': 0.5304125836693757, 'f': 0.3913194429465467}}
rouge_scores_2 :  {'rouge-1': {'r': 0.29780725851085577, 'p': 0.5139909664745728, 'f': 0.3705142922489634}, 'rouge-2': {'r': 0.13916482479247913, 'p': 0.2790610256859284, 'f': 0.18196432571744683}, 'rouge-l': {'r': 0.2812191325058027, 'p': 0.4868538812815655, 'f': 0.35014426779792696}}
ref_row :  ['zero_shot_multi_response_df', 0.31872592139524575, 0.5419428495104851, 0.39542450666013584, 0.15145548262645497, 0.2961012191639207, 0.19656614316750204, 0.2987575077719555, 0.5086332324754705, 0.37073185537223685]
Response_cols :  ['zero_shot_response_1', 'zero_shot_response_2']
technique_name :  zer

In [36]:
AllRouge_Scores

,Prompt Engineering Technique,rouge-1-r,rouge-1-p,rouge-1-f,rouge-2-r,rouge-2-p,rouge-2-f,rouge-l-r,rouge-l-p,rouge-l-f
0,few3_shot_single_response_df,0.268780,0.558806,0.360100,0.107240,0.280113,0.153755,0.254021,0.527766,0.340208
1,few3_shot_multi_response_df,0.249069,0.534411,0.337811,0.099096,0.255564,0.141630,0.234760,0.503478,0.318311
2,zero_shot_single_response_df,0.321041,0.562469,0.403883,0.151609,0.306312,0.199726,0.308085,0.539789,0.387611
3,zero_shot_multi_response_df,0.318726,0.541943,0.395425,0.151455,0.296101,0.196566,0.298758,0.508633,0.370732


#### Getting BERTScore

In [37]:
import statistics

In [38]:
bertscore = load("bertscore")

In [40]:
## BERT Scores
AllBERT_Scores = pd.DataFrame()
AllBERT_Scores = pd.DataFrame(columns=['Prompt Engineering Technique','Precision Mean','Precision Stddev','Recall Mean','Recall Stddev','F1 Score Mean','F1 Score Stddev','hashcode'])

# llm_model = ['phi3:latest','llama3.1:latest']
for response_df_index in range(len(response_df_list)): 
    Response_cols = list(filter(lambda x: (('score' not in x) and ('shot_response' in x)) == True, list(response_df_list[response_df_index].columns)))
    print('Response_cols : ',Response_cols)
    ModelSummary_col = 'model_response'
    technique_name = response_df_list_names[response_df_index]
    print('technique_name : ',technique_name)
    
    
    response_df_counter=1
    for Response_col in Response_cols:
        
        if response_df_counter ==1:
            test_bert_df = response_df_list[response_df_index][[Response_col,ModelSummary_col]].copy(deep=True)
            test_bert_df[Response_col] = test_bert_df[Response_col].replace('', np.nan)
            test_bert_df = test_bert_df.dropna(subset=[Response_col])
            test_bert_df = test_bert_df.reset_index(drop=True)
            
            BERT_Scores_1 = bertscore.compute(predictions=test_bert_df[Response_col], references=test_bert_df[ModelSummary_col], lang="en")
            precision_mean_1 = statistics.mean(BERT_Scores_1['precision'])
            precision_stdev_1 = statistics.stdev(BERT_Scores_1['precision'])
            recall_mean_1 = statistics.mean(BERT_Scores_1['recall'])
            recall_stdev_1 = statistics.stdev(BERT_Scores_1['recall'])
            f1_mean_1 = statistics.mean(BERT_Scores_1['f1'])
            f1_stdev_1 = statistics.stdev(BERT_Scores_1['f1'])
            hash_code_1 = BERT_Scores_1['hashcode']
        else:
            test_bert_df = response_df_list[response_df_index][[Response_col,ModelSummary_col]].copy(deep=True)
            test_bert_df[Response_col] = test_bert_df[Response_col].replace('', np.nan)
            test_bert_df = test_bert_df.dropna(subset=[Response_col])
            test_bert_df = test_bert_df.reset_index(drop=True)
            
            BERT_Scores_2 = bertscore.compute(predictions=test_bert_df[Response_col], references=test_bert_df[ModelSummary_col], lang="en")
            precision_mean_2 = statistics.mean(BERT_Scores_2['precision'])
            precision_stdev_2 = statistics.stdev(BERT_Scores_2['precision'])
            recall_mean_2 = statistics.mean(BERT_Scores_2['recall'])
            recall_stdev_2 = statistics.stdev(BERT_Scores_2['recall'])
            f1_mean_2 = statistics.mean(BERT_Scores_2['f1'])
            f1_stdev_2 = statistics.stdev(BERT_Scores_2['f1'])
            hash_code_2 = BERT_Scores_2['hashcode']
        response_df_counter+=1
        
    
    print('BERT_Scores_1 : ',BERT_Scores_1)
    print('BERT_Scores_2 : ',BERT_Scores_2)
    
    ref_row = [technique_name , 
               (precision_mean_1 + precision_mean_2)/2,
               (precision_stdev_1 + precision_stdev_2)/2,
               (recall_mean_1 + recall_mean_2)/2,
               (recall_stdev_1 + recall_stdev_2)/2,
               (f1_mean_1 + f1_mean_2)/2,
               (f1_stdev_1 + f1_stdev_2)/2,
               hash_code_1
               ]
    
    print('ref_row : ',ref_row)
    
    AllBERT_Scores.loc[-1] = ref_row
    AllBERT_Scores.index = AllBERT_Scores.index + 1  #shift index
    AllBERT_Scores = AllBERT_Scores.sort_index()  #sort by index
    

Response_cols :  ['zero_shot_response_1', 'zero_shot_response_2']
technique_name :  zero_shot_multi_response_df


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT_Scores_1 :  {'precision': [0.8601458072662354, 0.8855227828025818, 0.8416480422019958, 0.8959779739379883, 0.8832517862319946], 'recall': [0.8518903851509094, 0.8721424341201782, 0.8421362638473511, 0.8741439580917358, 0.8786553144454956], 'f1': [0.8559982180595398, 0.8787816762924194, 0.8418920636177063, 0.8849263191223145, 0.8809475302696228], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.48.2)'}
BERT_Scores_2 :  {'precision': [0.8225721716880798, 0.8939015865325928, 0.8606559038162231, 0.9065648913383484, 0.8881932497024536], 'recall': [0.8086493611335754, 0.8794127106666565, 0.857343316078186, 0.8706477284431458, 0.8741259574890137], 'f1': [0.8155513405799866, 0.8865979909896851, 0.8589964509010315, 0.8882433772087097, 0.8811034560203552], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.48.2)'}
ref_row :  ['zero_shot_multi_response_df', 0.8738434195518494, 0.02773806815710475, 0.8609147429466247, 0.022331710779502918, 0.8673038423061371, 0.0

In [41]:
AllBERT_Scores

,Prompt Engineering Technique,Precision Mean,Precision Stddev,Recall Mean,Recall Stddev,F1 Score Mean,F1 Score Stddev,hashcode
0,few3_shot_single_response_df,0.876893,0.018183,0.854730,0.006222,0.865609,0.010864,roberta-large_L17_no-idf_version=0.3.12(hug_tr...
1,few3_shot_multi_response_df,0.866608,0.027958,0.848216,0.014459,0.857243,0.020253,roberta-large_L17_no-idf_version=0.3.12(hug_tr...
2,zero_shot_single_response_df,0.876450,0.027754,0.863215,0.022342,0.869766,0.024775,roberta-large_L17_no-idf_version=0.3.12(hug_tr...
3,zero_shot_multi_response_df,0.873843,0.027738,0.860915,0.022332,0.867304,0.024628,roberta-large_L17_no-idf_version=0.3.12(hug_tr...
